In [1]:
using Pkg
Pkg.activate("..")

  Activating project at `~/IdeaProjects/neural-network-convolution-autodiff-libs/LIB-IMPL`


In [2]:
using JLD2
X_train = load("../data/KM3/imdb_dataset_prepared.jld2", "X_train")
y_train = load("../data/KM3/imdb_dataset_prepared.jld2", "y_train")
X_test = load("../data/KM3/imdb_dataset_prepared.jld2", "X_test")
y_test = load("../data/KM3/imdb_dataset_prepared.jld2", "y_test")
embeddings = load("../data/KM3/imdb_dataset_prepared.jld2", "embeddings")
vocab = load("../data/KM3/imdb_dataset_prepared.jld2", "vocab")
nothing

embedding_dim = size(embeddings,1)

50

In [3]:
using MYMLP

model = Chain(
    Embedding(length(vocab), embedding_dim),
    x->permutedims(x, (2,1,3)),
    # Conv((3,), embedding_dim => 8, relu),
    MaxPool((8,)),
    flatten,
    Dense(800, 1, sigmoid)
)
nothing

[ Info: Precompiling MYMLP [3cc38821-e486-4839-8a60-8da7bc74b6b2] (cache misses: include_dependency fsize change (2), mismatched flags (18))


In [4]:
using Printf, Statistics

dataset = DataLoader((X_train, y_train), batchsize=64, shuffle=true)

loss(m, x, y) = binarycrossentropy(m(x), y)
accuracy(m, x, y) =  mean((m(x).output .> 0.5) .== (y .> 0.5))

opt = setup(Adam(), model)

epochs = 5
for epoch in 1:epochs
    total_loss = 0.0
    total_acc = 0.0
    num_samples = 0

    t = @elapsed begin
        for (x, y) in dataset
            grads = gradient!(model) do m
                l = loss(m, x, y)
                total_loss += l.output
                total_acc += accuracy(model, x, y)
                return l
            end
            update!(opt, model, grads[1])
            num_samples += 1
        end

        train_loss = total_loss / num_samples
        train_acc = total_acc / num_samples

        test_acc = accuracy(model, X_test, y_test)
        test_loss = loss(model, X_test, y_test).output
    end

    println(@sprintf("Epoch: %d (%.2fs) \tTrain: (l: %.2f, a: %.2f) \tTest: (l: %.2f, a: %.2f)", 
        epoch, t, train_loss, train_acc, test_loss, test_acc))
end

Epoch: 1 (25.39s) 	Train: (l: 0.72, a: 0.55) 	Test: (l: 0.68, a: 0.59)
Epoch: 2 (13.51s) 	Train: (l: 0.60, a: 0.67) 	Test: (l: 0.58, a: 0.69)
Epoch: 3 (16.50s) 	Train: (l: 0.49, a: 0.77) 	Test: (l: 0.49, a: 0.77)
Epoch: 4 (18.28s) 	Train: (l: 0.40, a: 0.83) 	Test: (l: 0.44, a: 0.79)
Epoch: 5 (17.69s) 	Train: (l: 0.33, a: 0.86) 	Test: (l: 0.41, a: 0.82)
